In [8]:
#importing all the necessary libraries and nueral network model
# 1== correct 0==incorrect
import cv2 
import mediapipe as mp
import numpy as np
import os
import tensorflow as tf
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
# Load your trained neural network model (assuming "my_best_model.keras" is the checkpoint file)
model = tf.keras.models.load_model("my_best_model.keras")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 10)                  │             180 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              55 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 725 (2.84 KB)

 Trainable params: 241 (964.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 484 (1.89 KB)

In [9]:
#a dictionary to store the key points along with the anglaes to be calculated
# Define keypoints for each exercise
exercise_keypoints = {
    "plank": [
        ("LEFT_SHOULDER", "LEFT_ELBOW", "LEFT_WRIST"),
        ("RIGHT_SHOULDER", "RIGHT_ELBOW", "RIGHT_WRIST"),
        ("LEFT_HIP", "LEFT_KNEE", "LEFT_ANKLE"),
        ("RIGHT_HIP", "RIGHT_KNEE", "RIGHT_ANKLE"),
        ("LEFT_KNEE", "LEFT_ANKLE", "LEFT_HEEL"),
        ("RIGHT_KNEE", "RIGHT_ANKLE", "RIGHT_HEEL")
    ],
    "wall_sit": [
        ("LEFT_SHOULDER", "LEFT_HIP", "LEFT_KNEE"),
        ("RIGHT_SHOULDER", "RIGHT_HIP", "RIGHT_KNEE"),
        ("LEFT_HIP", "LEFT_KNEE", "LEFT_ANKLE"),
        ("RIGHT_HIP", "RIGHT_KNEE", "RIGHT_ANKLE"),
        ("LEFT_KNEE", "LEFT_ANKLE", "LEFT_HEEL"),
        ("RIGHT_KNEE", "RIGHT_ANKLE", "RIGHT_HEEL")
    ],
    "squat":[
             ('LEFT_SHOULDER', 'LEFT_HIP', 'LEFT_KNEE'),
             ('RIGHT_SHOULDER', 'RIGHT_HIP', 'RIGHT_KNEE'),
             ('LEFT_HIP', 'LEFT_KNEE', 'LEFT_ANKLE'),
             ('RIGHT_HIP', 'RIGHT_KNEE', 'RIGHT_ANKLE'),
    ]
    # Add more exercises and their keypoints here
    # "exercise_name": [("POINT_A", "POINT_B", "POINT_C"), ...]
}


In [10]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [11]:
Exercise = ['Exercise', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_HIP', 
'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE','LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
]
names=['plank','wall_sit','squat']


In [12]:
def calculate_exercise_angles(landmarks, keypoints, mp_pose, max_keypoints,current_exercise):
    angles = [0.0]*max_keypoints
    angles[0]=names.index(current_exercise)
    for points in keypoints:
        if all(landmarks[getattr(mp_pose.PoseLandmark, part).value].visibility > 0.4 for part in points):
            # Calculate angle based on the keypoints (this is just a placeholder, replace with actual calculation)
            a = [landmarks[getattr(mp_pose.PoseLandmark, points[0]).value].x, 
             landmarks[getattr(mp_pose.PoseLandmark, points[0]).value].y]
            b = [landmarks[getattr(mp_pose.PoseLandmark, points[1]).value].x, 
             landmarks[getattr(mp_pose.PoseLandmark, points[1]).value].y]
            c = [landmarks[getattr(mp_pose.PoseLandmark, points[2]).value].x, 
             landmarks[getattr(mp_pose.PoseLandmark, points[2]).value].y]
            angle=calculate_angle(a, b, c)
             # Get the index of the second keypoint name from the Exercise list
            idx = Exercise.index(points[1]) 
            print(idx)# Assuming the index is zero-based
            # Assign the calculated angle to the corresponding index in the angles list
            angles[idx] = angle
        
    # Pad with zeros to reach the max_keypoints length
    
    
    return angles


In [13]:
cap = cv2.VideoCapture(0)

# Open CSV file in append mode

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to capture image")
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
         
        # Extract landmarks
        if results.pose_landmarks:
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Specify the exercise
                current_exercise = "squat"  # Change this based on the exercise being performed
                
                # Get the keypoints for the current exercise
                keypoints = exercise_keypoints[current_exercise]
                
                # Calculate the angles
                angles = calculate_exercise_angles(landmarks, keypoints , mp_pose , 17,current_exercise)
                
                print("Angles:", angles)
                
                # Prepare the input for the neural network
                input_data = np.array(angles)
                input_data = np.expand_dims(input_data, axis=0)  # Shape it appropriately

                # Get the prediction from the neural network
                prediction = model.predict(input_data)
            

                # Determine if the exercise is correct based on the prediction
                if prediction[0] > 0.5:  # Adjust threshold as needed
                    exercise_status = "Correct"
                else:
                    exercise_status = "inCorrect"
                    
                # Display the prediction result on the image
                cv2.putText(image, exercise_status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")
                pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.7103039402735, 172.94271457832065, 177.93582921673539, 176.89327897548938, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.7089266037977, 173.600897236383, 177.02536864514013, 177.11050955945808, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.54935948801523, 174.27980514334848, 176.30049092053497, 176.9784464552817, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 173.0017537968791, 174.43773067463147, 174.84534951326495, 175.51805474654924, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 173.05989547956028, 175.85298270022855, 174.08931499690166, 174.23124434355074, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
7
8
9
10
A

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 178.8830597488468, 177.95486058093738, 172.61343283333855, 175.85632676365228, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 177.61768451285297, 178.37407398970632, 172.84577219540205, 174.4890204052231, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 176.99181193130616, 176.16719586676365, 173.45478288695784, 178.3771486794516, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 177.69390479894014, 176.15883330343195, 173.75051977653342, 177.98814931075782, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 177.41597772593607, 176.33634543883565, 173.55820863159443, 177.77827754871572, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 176.81363278888887, 178.61143606678377, 171.223599822401, 174.93287625631612, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 176.08624843047863, 178.70620148348115, 174.03683179595873, 175.28772257051924, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 175.27111409034478, 177.11723741574053, 176.0711825573851, 176.80290810527043, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 175.1510265100761, 176.10314450670893, 176.12716251930522, 177.45763998232832, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 173.73337015998914, 175.769858245221, 177.20970697923124, 177.5040419598323, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.75363779223193, 171.6824873445367, 175.6000112765013, 175.34503733495575, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.4811420466126, 172.47958655130725, 175.0713798901347, 175.10094936918435, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.4723864171079, 171.55185149402533, 174.881724845868, 175.9437218938295, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.31663283704188, 171.48275848403665, 175.0309005390743, 176.41665383651704, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 171.3880314495813, 171.9677347574262, 175.5547840961957, 176.04904686841067, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.15215499769084, 170.35854232959986, 173.93510056519102, 176.01176073434667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.22648056369044, 170.38831816872676, 173.9253051885823, 175.95844689123177, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.23012404108027, 170.24727987053348, 174.01264860906164, 176.18386732477694, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.29650814265406, 170.28463192277198, 173.94012234659266, 176.1256977737418, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.2947856962198, 170.24636614692926, 173.9561746710231, 176.18162895946347, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.39042600527148, 171.79954773830926, 176.08566102098297, 173.55179529582767, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.3329167471035, 172.85248823843185, 173.77232688981113, 171.28731630492024, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.41133226647764, 173.28423298153214, 172.9249925195873, 170.7879033775297, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.708679825277, 173.06755913525623, 173.1635829180435, 171.28102024122447, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.7887045767104, 172.96032641827603, 173.702767100127, 172.33989245899406, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 166.06521496624302, 169.69131222220793, 153.1310641814211, 154.10489803293748, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 173.28618296881962, 166.77034462226524, 144.98139696568842, 158.978666647273, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 171.47683504055115, 170.36395659619734, 173.28137663600143, 179.51338111447876, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 170.11004277882026, 174.20014629426805, 172.14824085700755, 176.78194402346088, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 174.37649385800123, 172.6330385146401, 173.13053703288773, 177.50224817301967, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 164.53135999453198, 161.79584967115568, 175.16041431530317, 171.57728306961528, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 164.95005036672424, 160.82853135957606, 175.0904143076064, 170.307506115643, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 165.51845852694072, 160.2828927595335, 175.5466029723414, 169.52040004415102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 165.68863906839334, 160.27424517702786, 175.62782847657695, 168.95229710344753, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 166.3839342610659, 161.1079416685331, 174.9015576132913, 169.13384568413719, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 155.64605358006432, 155.14476731693046, 158.5932558408883, 153.93256003244662, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 156.02253271774157, 154.9157164486421, 159.306642769551, 153.92581775587152, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 155.62972904314645, 155.02372072481836, 158.79007767440814, 153.92133098155313, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 155.83200306663602, 154.86105422417953, 159.0026231651767, 153.9263995239495, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 155.78403051720727, 154.91894076351335, 158.92849842298796, 154.14928895420087, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.0075888122701, 169.44576067164652, 176.43968583252814, 177.32975031191708, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.63605985437042, 170.24206497046035, 175.30895635912267, 175.92288744553295, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 172.2655185375478, 171.63984624919743, 174.06069899771776, 173.09994265497537, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 171.5886438104957, 171.52079101460532, 175.21500424551624, 173.40545078787824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
7
8
9
10
Angles: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 171.75831162077495, 171.15205025272132, 175.43967107902864, 173.70719793685998, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1/1 ━━━━